# Melhorias em implementação:

### Parte da classe atual ultrapassada:

In [1]:
from sgp4.api import SGP4_ERRORS

from astropy.coordinates import TEME, CartesianRepresentation  # CartesianDifferential
from astropy import units as u
from astropy.coordinates import ITRS

import pymap3d as pm
#
import pandas as pd
from sgp4 import omm
from sgp4.api import Satrec
from astropy.time import Time
from astropy.time import TimeDelta
import time
import numpy as np

class PropagInit:
    def __init__(self, satellite, lc, sample_time=1):
        self.satellite = satellite
        self.lc = lc
        self.sample_time = sample_time

        self.tempo = []
        self.traj = []
        self.dist = []
        self.hdmin = []
        self.dmin = []
        self.hrmin = []


    def orbit_propag(self, h0, n):
        time_array = np.linspace(h0,h0 + TimeDelta((n -1)* u.s),n)
        error_code, teme_p, teme_v = self.satellite.sgp4_array(time_array.jd1, time_array.jd2)
        teme_p = np.array(teme_p)
        x, y, z = teme_p[:,0], teme_p[:,1], teme_p[:,2]
        teme_p = CartesianRepresentation(x*u.km, y*u.km, z*u.km)
        teme = TEME(teme_p, obstime=time_array)  
        itrsp = teme.transform_to(ITRS(obstime=time_array))
        location = itrsp.earth_location
        enu_p = pm.ecef2enu(1000 * location.x.value,
                            1000 * location.y.value,
                            1000 * location.z.value, self.lc.lat,
                                    self.lc.lon, self.lc.height)

        enu_p = np.transpose(enu_p)
        enu_d = np.linalg.norm(enu_p,axis=1)
        min_index = np.argmin(enu_d)
        d_min = enu_d[min_index]
        posit_min = enu_p[min_index]
  
        self.tempo.append(time_array)
        self.traj.append(enu_p)
        self.dist.append(enu_d)
        self.hdmin.append(time_array[min_index])
        self.dmin.append(d_min)
        self.hrmin.append(min_index)
        return self

    def orbitpropag(self, t_inic, n):
        tvar = t_inic

        enu_dv = []
        tv = []
        enu_pv = []

        tsample = self.sample_time

        hr = 0

        flagmin = True
        hdmin = 0
        dmin = 0
        hrmin = 0

        for i in range(0, n):
            error_code, teme_p, teme_v = self.satellite.sgp4(tvar.jd1, tvar.jd2)
            if error_code != 0:
                # raise RuntimeError(SGP4_ERRORS[error_code])
                print(" SGP4_ERRORS: {}\n".format(SGP4_ERRORS[error_code]))
                break
            teme_p = CartesianRepresentation(teme_p * u.km)
            teme = TEME(teme_p, obstime=tvar)  # ver diferença entre UT1 e UTC
            itrsp = teme.transform_to(ITRS(obstime=tvar))
            location = itrsp.earth_location
            # location.geodetic
            enu_p = pm.ecef2enu(1000 * location.x.value, 1000 * location.y.value, 1000 * location.z.value, self.lc.lat,
                                self.lc.lon, self.lc.height)
            enu_d = (enu_p[0] ** 2 + enu_p[1] ** 2 + enu_p[2] ** 2) ** 0.5
            enu_dv.append(enu_d)
            enu_pv.append(enu_p)
            tv.append(tvar)

            if (enu_dv[len(enu_dv) - 2] < enu_dv[len(enu_dv) - 1]) and flagmin:
                print('NORAD: {} dmin= {:.3f}'.format(self.satellite.satnum, enu_dv[len(enu_dv) - 2]))
                dmin = enu_dv[len(enu_dv) - 2]
                hdmin = tvar - tsample / (60 * 60 * 24)
                hrmin = hr - 1
                flagmin = False

            hr += 1
            tvar += tsample / (60 * 60 * 24)
            if hr == n:
                self.tempo.append(tv)
                self.traj.append(enu_pv)
                self.dist.append(enu_dv)
                self.hdmin.append(hdmin)
                self.dmin.append(dmin)
                self.hrmin.append(hrmin)
        return self

    def searchh0(self, t_inic, t_final, distmin, distmin2,  n):

        enu_dv = []
        tv = []
        enu_pv = []
        flagmin = True
        ntraj = 0

        tvar = t_inic

        hr = 0
        hrmin = 0

        for i in range(0, n):
            error_code, teme_p, teme_v = self.satellite.sgp4(tvar.jd1, tvar.jd2)
            if error_code != 0:
                # raise RuntimeError(SGP4_ERRORS[error_code])
                print(" SGP4_ERRORS: {}\n".format(SGP4_ERRORS[error_code]))
                break
            teme_p = CartesianRepresentation(teme_p * u.km)
            teme = TEME(teme_p, obstime=tvar)  # ver diferença entre UT1 e UTC
            itrsp = teme.transform_to(ITRS(obstime=tvar))
            location = itrsp.earth_location
            # location.geodetic
            enu_p = pm.ecef2enu(1000 * location.x.value, 1000 * location.y.value, 1000 * location.z.value, self.lc.lat,
                                self.lc.lon, self.lc.height)
            enu_d = (enu_p[0]**2 + enu_p[1]**2 + enu_p[2]**2)**0.5

            if enu_d > distmin:
                tsample = round(1+abs((enu_d - distmin)/10000))
                print("sample = {}".format(tsample))
                if not flagmin:
                    flagmin = True
                    self.tempo.append(tv)
                    self.traj.append(enu_pv)
                    self.dist.append(enu_dv)
                    self.hdmin.append(hdmin)
                    self.dmin.append(dmin)
                    self.hrmin.append(hrmin)
                    tv = []
                    enu_pv = []
                    enu_dv = []
                hr = 0
                hrmin = 0
            else:
                tsample = self.sample_time
                tv.append(tvar)
                enu_dv.append(enu_d)
                enu_pv.append(enu_p)
                # print('{} {:.3f}'.format(tvar.value, enu_d))
                if (enu_dv[len(enu_dv)-2] < enu_dv[len(enu_dv)-1]) and flagmin:
                    dmin = enu_dv[len(enu_dv) - 2]
                    if dmin < distmin2:
                        hdmin = tvar - TimeDelta(tsample * u.s)
                        hrmin = hr - 1
                        flagmin = False
                        ntraj += 1
                        print('NORAD: {}, H0: {}, dmin= {:.3f}'.format(self.satellite.satnum, tv[0].value, dmin))
                    else:
                        tsample = 60*5

            hr += 1
            tvar += TimeDelta(tsample * u.s) # tsample  / (60 * 60 * 24)

            if tvar > t_final: # Time.strptime('2021-8-25 19:00:00', '%Y-%m-%d %H:%M:%S'):
                print("fim loop")
                break
        return self

    def searchh01(self, t_inic, t_final, distmin, distmin2,  n):

        enu_dv = []
        tv = []
        enu_pv = []
        flagmin = True
        ntraj = 0

        tvar = t_inic

        hr = 0
        hrmin = 0

        for i in range(0, n):
            error_code, teme_p, teme_v = self.satellite.sgp4(tvar.jd1, tvar.jd2)
            if error_code != 0:
                # raise RuntimeError(SGP4_ERRORS[error_code])
                print(" SGP4_ERRORS: {}\n".format(SGP4_ERRORS[error_code]))
                break
            teme_p = CartesianRepresentation(teme_p * u.km)
            teme = TEME(teme_p, obstime=tvar)  # ver diferença entre UT1 e UTC
            itrsp = teme.transform_to(ITRS(obstime=tvar))
            location = itrsp.earth_location
            # location.geodetic
            enu_p = pm.ecef2enu(1000 * location.x.value, 1000 * location.y.value, 1000 * location.z.value, self.lc.lat,
                                self.lc.lon, self.lc.height)
            enu_d = (enu_p[0]**2 + enu_p[1]**2 + enu_p[2]**2)**0.5

            # if enu_d < distmin:
            #     print('ok')
            if (enu_d - distmin)>0:
                tsample = round(5+abs(1*(enu_d - distmin)/8000))
            else:
                tsample = round(5+abs(2*(enu_d - distmin)/8000))            

            print("sample = {}, dist = {}".format(tsample,enu_d ))

            tvar += TimeDelta(tsample * u.s) # tsample  / (60 * 60 * 24)

            if tvar > t_final: # Time.strptime('2021-8-25 19:00:00', '%Y-%m-%d %H:%M:%S'):
                print("fim loop")
                break
        return self

    def searchh02(self, t_inic, t_final, distmin, distmin2,  n):

        enu_dv = []
        tv = []
        enu_pv = []
        flagmin = True
        ntraj = 0

        sign = 1

        tvar = t_inic

        hr = 0
        hrmin = 0

        for i in range(0, n):
            error_code, teme_p, teme_v = self.satellite.sgp4(tvar.jd1, tvar.jd2)
            if error_code != 0:
                # raise RuntimeError(SGP4_ERRORS[error_code])
                print(" SGP4_ERRORS: {}\n".format(SGP4_ERRORS[error_code]))
                break
            teme_p = CartesianRepresentation(teme_p * u.km)
            teme = TEME(teme_p, obstime=tvar)  # ver diferença entre UT1 e UTC
            itrsp = teme.transform_to(ITRS(obstime=tvar))
            location = itrsp.earth_location
            # location.geodetic
            enu_p = pm.ecef2enu(1000 * location.x.value, 1000 * location.y.value, 1000 * location.z.value, self.lc.lat,
                                self.lc.lon, self.lc.height)
            enu_d = (enu_p[0]**2 + enu_p[1]**2 + enu_p[2]**2)**0.5

            # if enu_d < distmin:
            #     print('ok')
            if (enu_d - distmin)>0:
                tsample = sign * round(5+abs(1*(enu_d - distmin)/8000))
            else:
                tvar += TimeDelta(60*10 * u.s)
                if sign > 0:  sign = -1
                else: sign = 1                              

            print("sample = {}, dist = {}".format(tsample,enu_d ))

            tvar += TimeDelta(tsample * u.s) # tsample  / (60 * 60 * 24)

            if tvar > t_final: # Time.strptime('2021-8-25 19:00:00', '%Y-%m-%d %H:%M:%S'):
                print("fim loop")
                break
        return self

    def pos_calc(self,tvar):
        error_code, teme_p, teme_v = self.satellite.sgp4(tvar.jd1, tvar.jd2)
        teme_p = CartesianRepresentation(teme_p * u.km)
        teme = TEME(teme_p, obstime=tvar)  # ver diferença entre UT1 e UTC
        itrsp = teme.transform_to(ITRS(obstime=tvar))
        location = itrsp.earth_location
        # location.geodetic
        enu_p = pm.ecef2enu(1000 * location.x.value, 1000 * location.y.value, 1000 * location.z.value, self.lc.lat,
                            self.lc.lon, self.lc.height)
        enu_d = (enu_p[0]**2 + enu_p[1]**2 + enu_p[2]**2)**0.5
        return enu_p, enu_d

class LocalFrame:
    def __init__(self, lat, lon, height):
        self.lat = lat
        self.lon = lon
        self.height = height

## Encontrar H0 atual:

In [46]:


orbital_elem = pd.read_csv("orbital_elem.csv").to_dict('records')
lc = LocalFrame(-5.923654, -35.167755, 50)

satellite = Satrec()
omm.initialize(satellite, orbital_elem[20])

posit = PropagInit(satellite,lc)


t_inic=Time('2022-08-11T00:00:00.000', format='isot', scale='utc')     # momento de inicio da busca
t_final= Time('2022-08-18T00:00:00.000', format='isot', scale='utc')   # momento de parar a busca

ini = time.time()
posit.searchh0(t_inic, t_final, 1100*1000,  1000*1000,  10000)
fim = time.time()
print("Tempo: ", fim - ini)

sample = 1166
sample = 1074
sample = 654
sample = 325
sample = 207
sample = 189
sample = 223
sample = 307
sample = 457
sample = 689
sample = 982
sample = 1186
sample = 1023
sample = 568
sample = 263
sample = 157
sample = 134
sample = 148
sample = 194
sample = 284
sample = 436
sample = 672
sample = 975
sample = 1196
sample = 1036
sample = 554
sample = 225
sample = 104
sample = 64
sample = 51
sample = 48
sample = 53
sample = 65
sample = 88
sample = 129
sample = 199
sample = 317
sample = 506
sample = 782
sample = 1092
sample = 1200
sample = 849
sample = 367
sample = 128
sample = 47
sample = 18
sample = 8
sample = 4
sample = 2
NORAD: 22015, H0: 2022-08-11T05:57:25.000, dmin= 679654.227
sample = 1
sample = 2
sample = 3
sample = 4
sample = 7
sample = 11
sample = 17
sample = 27
sample = 43
sample = 70
sample = 115
sample = 189
sample = 312
sample = 507
sample = 789
sample = 1102
sample = 1199
sample = 830
sample = 354
sample = 125
sample = 47
sample = 20
sample = 9
sample = 4
sample = 2
NORAD

In [ ]:
posit.tempo

In [4]:
orbital_elem[20]['TLE_LINE1']

'1 22015U 92039B   22223.28147519  .00006192  00000-0  22215-3 0  9994'

In [5]:
orbital_elem[20]['TLE_LINE2']

'2 22015  20.7129 189.9538 0042459  90.0946 270.4360 15.23545436676838'

## Melhoria Encontrar H0 rápido:

In [45]:
# Para Editar:

import numpy as np
from scipy.signal import argrelextrema

orbital_elem = pd.read_csv("orbital_elem.csv").to_dict('records')
lc = LocalFrame(-5.923654, -35.167755, 50)

t_inic=Time('2022-08-11T00:00:00.000', format='isot', scale='utc')     # momento de inicio da busca
t_final= Time('2022-08-18T00:00:00.000', format='isot', scale='utc')   # momento de parar a busca

sample_fast = 30
dist_max =1100000

def calc(orbital_elem, init,end, sample_time):    
    satellite = Satrec()
    omm.initialize(satellite, orbital_elem)
    pts = (end -init)/TimeDelta(sample_time * u.s)
    time_array = np.linspace(init, end ,int(pts.value)+1)
    #print(time_array.shape )
    error_code, teme_p, teme_v = satellite.sgp4_array(time_array.jd1, time_array.jd2)
    x, y, z = teme_p[:,0], teme_p[:,1], teme_p[:,2]
    teme_p = CartesianRepresentation(x*u.km, y*u.km, z*u.km)
    teme = TEME(teme_p, obstime=time_array)  
    itrsp = teme.transform_to(ITRS(obstime=time_array))
    location = itrsp.earth_location
    enu_p = pm.ecef2enu(1000 * location.x.value, 1000 * location.y.value, 1000 * location.z.value, lc.lat,
                                lc.lon, lc.height)
    
    enu_p = np.transpose(enu_p) 
    enu_d = np.linalg.norm(enu_p,axis=1)
    #print(enu_d )
    return time_array, enu_p, enu_d

ini = time.time()

time_array, enu_p, enu_d = calc(orbital_elem[20], t_inic, t_final, sample_fast)


dist_dif = enu_d - dist_max # angulo de 30 no ref ENU, 0.5 = sin(30)
idx1 = np.where(dist_dif[:-1] * dist_dif[1:] < 0 )[0] +1
tim_s = time_array[idx1]
# print(dist_dif[357:363])
# print(idx1)
# print(tim_s.value)

fim = time.time()
print("Tempo: ", fim - ini)

ini = time.time()

i = 0
while i < len(idx1)-1:
    if dist_dif[idx1[i]]<0:
        #print( 'H0: {}, HF: {}, d: {} \n'.format(time_array[idx1[i]], time_array[idx1[i+1]], enu_d[idx1[i]] ))
        time_array1, enu_p, enu_d = calc(orbital_elem[20],time_array[idx1[i]]-TimeDelta(sample_fast*u.s),time_array[idx1[i+1]], 1)
        print( 'H0: {}, HF: {}, dmin: {}, d0: {}, df: {} \n'.format(time_array1[0], time_array1[-1], np.min(enu_d),enu_d[0],enu_d[-1] ))
        i+=2
    else: i+=1
    
#     index = argrelextrema(enu_d, np.less)
#     enu_d_min =  enu_d[index]
#     idx_d_traj = np.where(enu_d < 1100000)[0]
#     traj = enu_d[idx_d_traj]

fim = time.time()
print("Tempo: ", fim - ini)

# #print(traj)
# print(enu_d)


Tempo:  0.020075321197509766
H0: 2022-08-11T05:57:00.000, HF: 2022-08-11T06:02:00.000, dmin: 679658.0999996277, d0: 1234401.7182242298, df: 1235424.5691903427 

H0: 2022-08-11T07:38:00.000, HF: 2022-08-11T07:42:30.000, dmin: 732703.1237662879, d0: 1123393.69244471, df: 1236454.1642980007 

H0: 2022-08-11T16:03:00.000, HF: 2022-08-11T16:08:00.000, dmin: 542803.2534035787, d0: 1138728.710287022, df: 1188134.9569160033 

H0: 2022-08-11T17:44:30.000, HF: 2022-08-11T17:47:30.000, dmin: 969138.2105127007, d0: 1143605.1874268171, df: 1154644.410110509 

H0: 2022-08-12T05:32:00.000, HF: 2022-08-12T05:36:30.000, dmin: 629027.5940855572, d0: 1144300.034177423, df: 1100973.6551343517 

H0: 2022-08-12T07:12:30.000, HF: 2022-08-12T07:17:00.000, dmin: 774242.2318249625, d0: 1249822.6939903887, df: 1163064.4177205889 

H0: 2022-08-12T15:37:30.000, HF: 2022-08-12T15:43:00.000, dmin: 527288.1422838406, d0: 1240599.3254454734, df: 1254407.7881002338 

H0: 2022-08-12T17:19:30.000, HF: 2022-08-12T17:22:00

In [89]:
from skyfield.api import load, wgs84

tles1 = load.tle_file('conftle.txt')

#tles2 = load.tle_file()

sat = tles1[0]
print(sat)

ini = time.time()

ts = load.timescale()

bluffton = wgs84.latlon(-5.92256, -35.1615)
t0 = ts.utc(2022, 8, 11)
t1 = ts.utc(2022, 8, 18)
t, events = sat.find_events(bluffton, t0, t1, altitude_degrees=30.0)

fim = time.time()
print("Tempo: ", fim - ini)

ini = time.time()
for ti, event in zip(t, events):
    name = ('rise above 30°', 'culminate', 'set below 30°')[event]
    print(ti.utc_strftime('%Y-%m-%dT%H:%M:%S'), name)

    tem = Time(ti.utc_strftime('%Y-%m-%dT%H:%M:%S'), format='isot')
    enup, enud = posit.pos_calc(tem) ## Lento
    print(enud)
fim = time.time()
print("numero: ", events.shape )
print("Tempo teme to enu: ", fim - ini)

catalog #22015 epoch 2022-08-11 06:45:19 UTC
Tempo:  0.017378568649291992
2022-08-11T05:58:05 rise above 30°
895977.0351996033
2022-08-11T05:59:31 culminate
679698.8065682248
2022-08-11T06:00:58 set below 30°
910673.8295055908
2022-08-11T07:38:43 rise above 30°
920549.431502886
2022-08-11T07:40:05 culminate
732715.5971286349
2022-08-11T07:41:28 set below 30°
929805.09690544
2022-08-11T16:03:37 rise above 30°
923970.8689106569
2022-08-11T16:05:26 culminate
542792.3972932128
2022-08-11T16:07:14 set below 30°
918957.1202116479
2022-08-12T05:32:48 rise above 30°
887073.4096373181
2022-08-12T05:34:20 culminate
629083.227991606
2022-08-12T05:35:52 set below 30°
898984.6363285336
2022-08-12T07:13:43 rise above 30°
911602.8536278032
2022-08-12T07:14:54 culminate
774257.9578191183
2022-08-12T07:16:06 set below 30°
920895.2144702657
2022-08-12T15:38:22 rise above 30°
931099.4777526197
2022-08-12T15:40:14 culminate
527288.1422838406
2022-08-12T15:42:05 set below 30°
926310.1782350001
2022-08-13T0

## Melhoria Código rápido de calculo trajetória:

In [95]:


tem = Time(t.utc_strftime('%Y-%m-%dT%H:%M:%S'), format='isot')

ini = time.time()

error_code, teme_p, teme_v = satellite.sgp4_array(tem.jd1, tem.jd2)

teme_p = np.array(teme_p)
x, y, z = teme_p[:,0], teme_p[:,1], teme_p[:,2]

teme_p = CartesianRepresentation(x*u.km, y*u.km, z*u.km)
teme = TEME(teme_p, obstime=tem)  # ver diferença entre UT1 e UTC
itrsp = teme.transform_to(ITRS(obstime=tem))
location = itrsp.earth_location
enu_p = pm.ecef2enu(1000 * location.x.value, 1000 * location.y.value, 1000 * location.z.value, lc.lat,
                            lc.lon, lc.height)

fim = time.time()
print("Tempo: ", fim - ini)

enu_p

Tempo:  0.003001689910888672


(array([   25466.06337329,  -234535.48500898,  -485454.81320164,
         -878175.99713957, -1024634.30626479, -1164614.57977876,
          780143.46398059,   540860.39993516,   290103.4642783 ]),
 array([ 1216598.09923688,    51033.27412469, -1116505.87515824,
         -843261.56259651,  -250690.24273257,   343486.73060408,
          927915.67102392,  -127756.80189316, -1188335.8860551 ]),
 array([700461.93302992, 802338.92532322, 704343.79299177, 704159.42803241,
        729241.30021246, 702741.30093862, 701815.80867844, 785162.04318144,
        703506.06784671]))

### Funcionando

In [262]:
import numpy as np
orbital_elem = pd.read_csv("traj_data.csv").to_dict('records')
lc = LocalFrame(-5.923654, -35.167755, 50)

usecols = ['NORAD_CAT_ID', 'H0', 'N_PT']
df_conf = pd.read_csv("traj_data.csv",usecols=usecols)

# df_conf = pd.read_csv("traj_data.csv")

ini = time.time()

for index, row in df_conf.iterrows():
    satellite = Satrec()
    omm.initialize(satellite, orbital_elem[index])
    h0 = Time(row['H0'], format='isot')
    time_array = np.linspace(h0,h0 + TimeDelta((row['N_PT'] -1)* u.s),row['N_PT'])

    error_code, teme_p, teme_v = satellite.sgp4_array(time_array.jd1, time_array.jd2)
    teme_p = np.array(teme_p)
    x, y, z = teme_p[:,0], teme_p[:,1], teme_p[:,2]

    teme_p = CartesianRepresentation(x*u.km, y*u.km, z*u.km)
    teme = TEME(teme_p, obstime=time_array)  
    itrsp = teme.transform_to(ITRS(obstime=time_array))
    location = itrsp.earth_location
    enu_p = pm.ecef2enu(1000 * location.x.value, 1000 * location.y.value, 1000 * location.z.value, lc.lat,
                                lc.lon, lc.height)

    enu_p = np.transpose(enu_p)
    enu_d = np.linalg.norm(enu_p,axis=1)
    min_index = np.argmin(enu_d)
    d_min = enu_d[min_index]
    posit_min = enu_p[min_index]
    
    break

fim = time.time()
print("Tempo: ", fim - ini)

# print(enu_d)



Tempo:  0.0039975643157958984


In [263]:
np.shape(enu_p)

(233, 3)

In [151]:
import numpy as np
orbital_elem = pd.read_csv("traj_data.csv").to_dict('records')
lc = LocalFrame(-5.923654, -35.167755, 50)

usecols = ['NORAD_CAT_ID', 'H0', 'N_PT']
df_conf = pd.read_csv("traj_data.csv",usecols=usecols)

# df_conf = pd.read_csv("traj_data.csv")

ini = time.time()

for index, row in df_conf.iterrows():
    satellite = Satrec()
    orbital_elem_row = next(x for x in orbital_elem if x["NORAD_CAT_ID"] == row['NORAD_CAT_ID'])
    omm.initialize(satellite, orbital_elem_row)
    propag = PropagInit(satellite, lc, 1) 
    pos = propag.orbitpropag(Time(row['H0'], format='isot'),row['N_PT'])
    
    break

fim = time.time()
print("Tempo: ", fim - ini)


NORAD: 2142 dmin= 776821.976
Tempo:  0.39698338508605957


In [232]:
import numpy as np
orbital_elem = pd.read_csv("traj_data.csv").to_dict('records')
lc = LocalFrame(-5.923654, -35.167755, 50)

usecols = ['NORAD_CAT_ID', 'H0', 'N_PT']
df_conf = pd.read_csv("traj_data.csv",usecols=usecols)

# df_conf = pd.read_csv("traj_data.csv")

ini = time.time()
sdf = SummarizeDataFiles()
for index, row in df_conf.iterrows():
    satellite = Satrec()
    orbital_elem_row = next(x for x in orbital_elem if x["NORAD_CAT_ID"] == row['NORAD_CAT_ID'])
    omm.initialize(satellite, orbital_elem_row)
    propag = PropagInit(satellite, lc, 1) 
    pos = propag.orbit_propag(Time(row['H0'], format='isot'),row['N_PT'])
    sdf.save_trajectories(pos,orbital_elem_row)
    break

fim = time.time()
print("Tempo: ", fim - ini)

Tempo:  0.0039997100830078125


In [200]:
np.shape(pos.dist )


(1, 3)

In [231]:
class SummarizeDataFiles:
    def __init__(self):
        self.sel_orbital_elem = []
        self.sel_resume = { "H0":[], "DIST_H0":[],"H_DIST_MIN":[],"PT_DIST_MIN":[],
                    "DIST_MIN":[],"HF":[], "N_PT":[], "DIST_HF":[],"RCS":[] }

    def save_trajectories(self,pos,orbital_elem): 
        for i in range(0, len(pos.traj)):
            tempo = pos.tempo[i]
            posenu = pos.traj[i]
            distenu = pos.dist[i]
            
            ttxt = tempo[0].strftime('%Y_%m_%d-H0-%H_%M_%S')

            # Summarized data set of the trajectories obtained
            self.sel_orbital_elem.append(orbital_elem) 

            self.sel_resume["RCS"].append(orbital_elem['RCS_SIZE'])
            self.sel_resume["H0"].append(tempo[0].value)
            self.sel_resume["DIST_H0"].append(distenu[0])
            self.sel_resume["H_DIST_MIN"].append(pos.hdmin[i].value[11:])
            self.sel_resume["PT_DIST_MIN"].append(pos.hrmin[i])
            self.sel_resume["DIST_MIN"].append(pos.dmin[i])
            self.sel_resume["HF"].append(tempo[len(tempo) - 1].value[11:])
            self.sel_resume["N_PT"].append(len(distenu) - 1)
            self.sel_resume["DIST_HF"].append(distenu[len(distenu) - 1]) 

In [233]:
sdf.sel_resume

{'H0': ['2022-08-06T12:30:39.000'],
 'DIST_H0': [1070257.5070111575],
 'H_DIST_MIN': ['12:32:39.000'],
 'PT_DIST_MIN': [120],
 'DIST_MIN': [11266.620239357559],
 'HF': ['12:34:31.000'],
 'N_PT': [232],
 'DIST_HF': [1261516.725177206],
 'RCS': ['LARGE']}

## Procura automática de H0 Backup

In [ ]:
# Para Editar:

import numpy as np
from scipy.signal import argrelextrema

orbital_elem = pd.read_csv("orbital_elem.csv").to_dict('records')
lc = LocalFrame(-5.923654, -35.167755, 50)

t_inic=Time('2022-08-11T00:00:00.000', format='isot', scale='utc')     # momento de inicio da busca
t_final= Time('2022-08-18T00:00:00.000', format='isot', scale='utc')   # momento de parar a busca

sample_fast = 60
dist_max =1100000

def calc(orbital_elem, init,end, sample_time):    
    satellite = Satrec()
    omm.initialize(satellite, orbital_elem)
    pts = (end -init)/TimeDelta(sample_time * u.s)
    time_array = np.linspace(init, end ,int(pts.value)+1)
    print(time_array.shape )
    error_code, teme_p, teme_v = satellite.sgp4_array(time_array.jd1, time_array.jd2)
    x, y, z = teme_p[:,0], teme_p[:,1], teme_p[:,2]
    teme_p = CartesianRepresentation(x*u.km, y*u.km, z*u.km)
    teme = TEME(teme_p, obstime=time_array)  
    itrsp = teme.transform_to(ITRS(obstime=time_array))
    location = itrsp.earth_location
    enu_p = pm.ecef2enu(1000 * location.x.value, 1000 * location.y.value, 1000 * location.z.value, lc.lat,
                                lc.lon, lc.height)
    
    enu_p = np.transpose(enu_p) 
    enu_d = np.linalg.norm(enu_p,axis=1)
    #print(enu_d )
    return time_array, enu_p, enu_d

ini = time.time()

time_array, enu_p, enu_d = calc(orbital_elem[20], t_inic, t_final, sample_fast)

# satellite = Satrec()
# omm.initialize(satellite, orbital_elem[20])


# time_array = np.linspace(t_inic,t_final,int(pts.value)+1 )
# #print(time_array.value )
# error_code, teme_p, teme_v = satellite.sgp4_array(time_array.jd1, time_array.jd2)
# x, y, z = teme_p[:,0], teme_p[:,1], teme_p[:,2]
# teme_p = CartesianRepresentation(x*u.km, y*u.km, z*u.km)
# teme = TEME(teme_p, obstime=time_array)  
# itrsp = teme.transform_to(ITRS(obstime=time_array))
# location = itrsp.earth_location
# enu_p = pm.ecef2enu(1000 * location.x.value, 1000 * location.y.value, 1000 * location.z.value, lc.lat,
#                              lc.lon, lc.height)

#enu_p = np.transpose(enu_p)
# idx_z_pos = np.where(enu_p[:,2] > 0)[0]
# #print(idx_z_pos)
# enu_p_z = enu_p[idx_z_pos]
# #print(enu_p_z)
# enu_d = np.linalg.norm(enu_p_z,axis=1)
# idx_d = np.where(enu_d < 5000000)[0]
# #print(enu_d[idx_d])
# enu_d5 = enu_d[idx_d]
# index = argrelextrema(enu_d5, np.less)
# print(enu_d[index])

# enu_p = np.transpose(enu_p) 
# enu_d = np.linalg.norm(enu_p,axis=1)
# #print(enu_d)
# index = argrelextrema(enu_d, np.less)
# enu_d_min =  enu_d[index]
# idx_d_val = np.where(enu_d_min < 1100000)[0]

# enu_p = np.transpose(enu_p) 
# enu_d = np.linalg.norm(enu_p,axis=1)
# index = argrelextrema(enu_d, np.less)
# enu_d_min =  enu_d[index]
# tim_s = time_array[index]
# idx_d_val = np.where((enu_d_min < 1*1100000)&(enu_d_min >0) )[0]

# enu_p = np.transpose(enu_p) 
# enu_d = np.linalg.norm(enu_p,axis=1)
# a = enu_p[:,2] - 0.5*enu_d # angulo de 30 no ref ENU, 0.5 = sin(30)
# idx1 = np.where(a[:-1] * a[1:] < 0 )[0] +1
# tim_s = time_array[idx1]
# print(a[358:362])
# print(idx1)

dist_dif = enu_d - dist_max # angulo de 30 no ref ENU, 0.5 = sin(30)
idx1 = np.where(dist_dif[:-1] * dist_dif[1:] < 0 )[0] +1
tim_s = time_array[idx1]
# print(dist_dif[357:363])
# print(idx1)
# print(tim_s.value)

# index_min = argrelextrema(enu_p[:,2], np.greater)
# index_in_ot = np.where((np.abs(enu_p[:,2]) < 1*1100000)&(enu_d_min >0) )[0]
# tim_s = time_array[index]
# idx_d_val = np.where((enu_d_min < 1*1100000)&(enu_d_min >0) )[0]

# time_selec = tim_s[idx_d_val]
fim = time.time()
print("Tempo: ", fim - ini)

ini = time.time()

i = 0
while i < len(idx1)-1:
    if dist_dif[idx1[i]]<0:
        #print( 'H0: {}, HF: {}, d: {} \n'.format(time_array[idx1[i]], time_array[idx1[i+1]], enu_d[idx1[i]] ))
        time_array1, enu_p, enu_d = calc(orbital_elem[20],time_array[idx1[i]],time_array[idx1[i+1]], 1)
        print( 'H0: {}, HF: {}, d: {} \n'.format(time_array1[0], time_array1[-1], np.min(enu_d) ))
        i+=2
    else: i+=1
    
#     enu_p = np.transpose(enu_p) 
#     enu_d = np.linalg.norm(enu_p,axis=1)
#     index = argrelextrema(enu_d, np.less)
#     enu_d_min =  enu_d[index]
#     idx_d_traj = np.where(enu_d < 1100000)[0]
#     traj = enu_d[idx_d_traj]

fim = time.time()
print("Tempo: ", fim - ini)

# #print(traj)
# print(enu_d)